# 13. 비지도학습_텍스트 마이닝

## 01. [감성 분석 + 토픽 모델링] 영화 리뷰 데이터로 감성 예측하기
- https://github.com/e9t/nsmc
- ratings_test.txt 다운로드
- label 컬럼은 감성 분류 클래스 값
- 1에서 10점의 평점 중에서 중립적인 평점인 5에서 8점은 제외하고 1에서 4점을 부정 감성 0으로, 9에서 10점을 긍정 감성 1로 표시

## 1부

In [1]:
import pandas as pd
pd.__version__

'1.4.2'

In [2]:
# 한글 UnicodeEncodingError 방지를 위해 기본 인코딩을 utf-8로 설정
import os
os.environ['PYTHONIOENCODING']='utf-8'

In [3]:
# 경고 메시지 표시 안하게 설정하기
import warnings
warnings.filterwarnings(action='ignore')

### 2-1) 평가용 데이터 준비
#### (1) 훈련용 데이터 파일 로드

In [4]:
nsmc_train_df=pd.read_csv('data/ratings_train.txt', encoding='utf-8', sep='\t', engine='python')
nsmc_train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


#### (2) 데이터 정보 확인

In [5]:
nsmc_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


#### (3) document 칼럼이 Null인 샘플 제거

In [6]:
nsmc_train_df = nsmc_train_df[nsmc_train_df['document'].notnull()]

In [7]:
nsmc_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


#### (4) 타겟 컬럼 label 확인(0:부정감석, 1:긍정감성)

In [8]:
nsmc_train_df['label'].value_counts()

0    75170
1    74825
Name: label, dtype: int64

#### (5) 한글 이외의 문자(특수문자 등)는 공백으로 변환(정규표현식 이용)

In [9]:
import re

In [10]:
nsmc_train_df['document']=nsmc_train_df['document'].apply(lambda x:re.sub(r'[^ ㄱ-ㅣ가-힣]+', " ", x))
nsmc_train_df.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


### 2-2) 평가용 데이터 준비
#### (1) 평가용 데이터 파일 로드

In [11]:
nsmc_test_df= pd.read_csv('data/ratings_test.txt', encoding='utf-8', sep='\t', engine='python')
nsmc_test_df.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


#### (2) 데이터 정보 확인

In [12]:
nsmc_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


#### (3) document 칼럼이 Null인 샘플 제거

In [13]:
nsmc_test_df = nsmc_test_df[nsmc_test_df['document'].notnull()]

#### (4) 타겟 컬럼 label 확인(0:부정감석, 1:긍정감성)

In [14]:
nsmc_test_df['label'].value_counts()

1    25171
0    24826
Name: label, dtype: int64

#### (5) 한글 이외의 문자(특수문자 등)는 공백으로 변환(정규표현식 이용)

In [15]:
nsmc_test_df['document']=nsmc_test_df['document'].apply(lambda x:re.sub(r'[^ ㄱ-ㅣ가-힣]+', " ", x))
nsmc_test_df.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,,0
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0


## 3. 분석 모델 구축
### 3-1) 피처 벡터화 : TF-IDF
#### (1) 형태소를 분석하여 토큰화 : 한글 형태소 엔진으로 Okt 이용

In [16]:
! pip install konlpy

In [17]:
from konlpy.tag import Okt
okt=Okt()

In [18]:
def okt_tokenizer(text):
    tokens=okt.morphs(text)
    return tokens

#### (2) TF-IDF 기반 피처 벡터 생성 : 실행시간 10분 정도 걸림

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer(tokenizer = okt_tokenizer, ngram_range = (1, 2), min_df = 3, max_df = 0.9)
tfidf.fit(nsmc_train_df['document'])
nsmc_train_tfidf = tfidf.transform(nsmc_train_df['document']) # tfidf 벡터 형태로 변환

### 3-2) 감성 분류 모델 구축 : 로지스틱 회귀를 이용한 이진 분류
### - Sentiment Analysis using Logistic Regression
#### (1) 로지스틱 회귀 기반 분석모델 생성

In [20]:
from sklearn.linear_model import LogisticRegression
SA_lr=LogisticRegression(random_state=0)

In [21]:
SA_lr.fit(nsmc_train_tfidf, nsmc_train_df['label'])

LogisticRegression(random_state=0)

#### (2) 로지스틱 회귀의 best 하이퍼파라미터 찾기

In [22]:
from sklearn.model_selection import GridSearchCV

params={'C':[1,3,3.5,4,4.5,5]}
SA_lr_grid_cv=GridSearchCV(SA_lr, param_grid=params, cv=3, scoring='accuracy', verbose=1)

#### (3) 최적 분석 모델 훈련

In [23]:
SA_lr_grid_cv.fit(nsmc_train_tfidf, nsmc_train_df['label'])

Fitting 3 folds for each of 6 candidates, totalling 18 fits


GridSearchCV(cv=3, estimator=LogisticRegression(random_state=0),
             param_grid={'C': [1, 3, 3.5, 4, 4.5, 5]}, scoring='accuracy',
             verbose=1)

In [24]:
print(SA_lr_grid_cv.best_params_, round(SA_lr_grid_cv.best_score_,4)) # 하이퍼파라미터에 따라 정확도 표현

{'C': 3} 0.8553


In [29]:
# 최적 파라미터의 best 모델 저장
SA_lr_best=SA_lr_grid_cv.best_estimator_

## 4. 분석 모델 평가
### 4-1) 평가용 데이터를 이용하여 감성 분석 모델 정확도

In [26]:
# 평가용 데이터의 피처 벡터화
nsmc_test_tfidf=tfidf.transform(nsmc_test_df['document'])

In [31]:
test_predict=SA_lr_best.predict(nsmc_test_tfidf)

### 4-2) 새로운 텍스트에 대한 감성 예측

In [34]:
st=input('감성 분석할 문장입력 >> ')

감성 분석할 문장입력 >>  웃자 ^o^ 오늘은 좋은 날이 될 것 같은 예감100%! ^^*


In [35]:
# 0) 입력 텍스트에 대한 전처리 수행
st = re.compile(r'[ㄱ-ㅣ가-힣]+').findall(st) ; print(st)
st = [" ".join(st)] ; print(st)

['웃자', '오늘은', '좋은', '날이', '될', '것', '같은', '예감']
['웃자 오늘은 좋은 날이 될 것 같은 예감']


In [36]:
#1) 입력 텍스트의 피처 벡터화 
st_tfidf = tfidf.transform(st)
#2) 최적 감성 분석 모델에 적용하여 감성 분석 평가 
st_predict = SA_lr_best.predict(st_tfidf)

In [37]:
#3) 예측값 출력하기
if(st_predict == 0): 
    print(st , "->> 부정 감성") 
else :
    print(st , "->> 긍정 감성")

['웃자 오늘은 좋은 날이 될 것 같은 예감'] ->> 긍정 감성


## 2부...